In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [20]:
data = spark.read.csv('/Users/trinakarmakar/downloads/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Natural_Language_Processing/smsspamcollection/smsspamcollection', 
                     inferSchema=True, sep='\t')

In [21]:
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

In [22]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [23]:
from pyspark.sql.functions import length

In [24]:
data = data.withColumn('length', length(data['text']))

In [25]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [26]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [32]:
from pyspark.ml.feature import (Tokenizer, StopWordsRemover,CountVectorizer,IDF, StringIndexer)

In [37]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token_text')
count_vec = CountVectorizer(inputCol='stop_token_text', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')

In [38]:
from pyspark.ml.feature import VectorAssembler

In [57]:
clean_up = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')

In [58]:
from pyspark.ml.classification import NaiveBayes

In [59]:
nb = NaiveBayes()

In [60]:
from pyspark.ml import Pipeline

In [61]:
data_prep_pipe = Pipeline()

In [62]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric, tokenizer,
                                 stop_remove, count_vec, idf,clean_up])

In [63]:
cleaner = data_prep_pipe.fit(data)

In [64]:
clean_data = cleaner.transform(data)

In [47]:
clean_data.columns

['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_token_text',
 'c_vec',
 'tf_idf',
 'feature']

In [65]:
clean_data = clean_data.select('label', 'features')

In [66]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [67]:
training, test = clean_data.randomSplit([0.7, 0.3])

In [68]:
spam_detector = nb.fit(training)

In [69]:
test_results = spam_detector.transform(test)

In [70]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[-792.14331516994...|[1.0,4.2978937961...|       0.0|
|  0.0|(13424,[0,1,2,41,...|[-1091.6853763010...|[1.0,1.7497676562...|       0.0|
|  0.0|(13424,[0,1,5,15,...|[-1004.7640390585...|[1.0,7.0290043022...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-537.81597707874...|[1.0,1.6688866944...|       0.0|
|  0.0|(13424,[0,1,15,20...|[-669.69944230997...|[1.0,1.7326192270...|       0.0|
|  0.0|(13424,[0,1,23,63...|[-1294.6445160418...|[1.0,1.5975905298...|       0.0|
|  0.0|(13424,[0,1,31,43...|[-339.40947041029...|[1.0,3.4187391321...|       0.0|
|  0.0|(13424,[0,1,46,17...|[-1137.0803799211...|[2.35977106429155...|       1.0|
|  0.0|(13424,[0,1,72,10...|[-669.06648624464...|[1.0,7.1076301130...|       0.0|
|  0.0|(13424,[0

In [72]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [73]:
acc_eval = MulticlassClassificationEvaluator()

In [76]:
acc = acc_eval.evaluate(test_results)

In [77]:
print('Acc of NB Model')
print(acc)

Acc of NB Model
0.9196606208087982
